In [ ]:
import hana_ml.dataframe as dataframe
from notebook_hana_connector.notebook_hana_connector import NotebookConnectionContext
import pandas as pd
import xgboost
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
#from xgboost import XGBRegressor
import sklearn
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import accuracy_score
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

In [ ]:
conn = NotebookConnectionContext(connectionId = 'hanapoc')

In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

Get 10000 COMPLETED rows from training dataset

In [ ]:
sql = (
"select "
"top 10000 "
"*, "
"(ORD_LINE_CANC / (ORD_LINE_COMP + ORD_LINE_CANC)) as ORD_CANC_RATIO, "
"(CUST_LINES_CANC / (CUST_LINES_COMP + CUST_LINES_CANC)) as CUST_CANC_RATIO, "
"(PLANT_LINES_CANC / (PLANT_LINES_COMP + PLANT_LINES_CANC)) as PLANT_CANC_RATIO "
"from VW_BWH_TRAIN_LINE_ORD_CUST_DEPOT "
"where NET_VALUE > '0' "
"and COMP_FLAG = '1' "
"ORDER BY RAND() LIMIT 10000"
)

po_sel = conn.sql(sql)
comp = po_sel.collect()
#comp

Get 10000 NOT-COMPLETED rows from training dataset

In [ ]:
sql = (
"select "
"top 10000 "
"*, "
"(ORD_LINE_CANC / (ORD_LINE_COMP + ORD_LINE_CANC)) as ORD_CANC_RATIO, "
"(CUST_LINES_CANC / (CUST_LINES_COMP + CUST_LINES_CANC)) as CUST_CANC_RATIO, "
"(PLANT_LINES_CANC / (PLANT_LINES_COMP + PLANT_LINES_CANC)) as PLANT_CANC_RATIO "
"from VW_BWH_TRAIN_LINE_ORD_CUST_DEPOT "
"where NET_VALUE > '0' "
"and COMP_FLAG = '0' "
"ORDER BY RAND() LIMIT 10000"
)

po_sel = conn.sql(sql)
delet = po_sel.collect()
#delet

Combine and format the two datasets

In [ ]:
combined_df = pd.concat([comp,delet])
combined_df.info()

In [ ]:
combined_df['NET_VALUE'] = combined_df['NET_VALUE'].astype(float).astype(int)
combined_df['ORD_NET_VALUE_TOTAL'] = combined_df['ORD_NET_VALUE_TOTAL'].astype(float).astype(int)
combined_df['ORD_NET_VALUE_COMP'] = combined_df['ORD_NET_VALUE_COMP'].astype(float).astype(int)
combined_df['ORD_NET_VALUE_CANC'] = combined_df['ORD_NET_VALUE_CANC'].astype(float).astype(int)
combined_df['CUST_NET_VALUE_TOTAL'] = combined_df['CUST_NET_VALUE_TOTAL'].astype(float).astype(int)
combined_df['CUST_NET_VALUE_COMP'] = combined_df['CUST_NET_VALUE_COMP'].astype(float).astype(int)
combined_df['CUST_NET_VALUE_CANC'] = combined_df['CUST_NET_VALUE_CANC'].astype(float).astype(int)
combined_df['PLANT_NET_VALUE_TOTAL'] = combined_df['PLANT_NET_VALUE_TOTAL'].astype(float).astype(int)
combined_df['PLANT_NET_VALUE_COMP'] = combined_df['PLANT_NET_VALUE_COMP'].astype(float).astype(int)
combined_df['PLANT_NET_VALUE_CANC'] = combined_df['PLANT_NET_VALUE_CANC'].astype(float).astype(int)
combined_df['ORD_CANC_RATIO'] = combined_df['ORD_CANC_RATIO'].astype(float)
combined_df['CUST_CANC_RATIO'] = combined_df['CUST_CANC_RATIO'].astype(float)
combined_df['PLANT_CANC_RATIO'] = combined_df['PLANT_CANC_RATIO'].astype(float)
#include region

In [ ]:
from sklearn.preprocessing import LabelBinarizer

In [ ]:
y = combined_df['COMP_FLAG']
data = combined_df.drop(['CANC_FLAG','COMP_FLAG','DOC_NUMBER','S_ORD_ITEM','CUSTOMER','CUST_NAME','CUST_CITY','CUST_POSTAL_CD','PLANT','PLANT_NAME','MATERIAL','ORDSTAT','CREATEDON_SO','CREATEDON_LINE','LINE_COMP_DATE','K8INV_DT','RECORDMODE'], axis=1)
#include region
#x.dtypes

Fit a label binarizer to region column to do one-hot-encoding

In [ ]:
jobs_encoder = LabelBinarizer()
jobs_encoder.fit(data['REGION'])
transformed = jobs_encoder.transform(data['REGION'])
ohe_df = pd.DataFrame(transformed, columns=jobs_encoder.classes_)
data.reset_index(inplace=True, drop=True)
x = pd.concat([data, ohe_df], axis=1)
x = x.drop(['REGION'], axis = 1)
x.head(15)

Fit a XGBoost classification model to the training dataset

In [ ]:
model = XGBClassifier(objective='binary:logistic', eval_metric='logloss', n_estimators=300, early_stopping_rounds=50, verbose_eval=50, learning_rate=0.005, max_depth=7)
#model = XGBClassifier(objective='binary:logistic', eval_metric='error@0.15', n_estimators=300, early_stopping_rounds=50, verbose_eval=50, learning_rate=0.005, max_depth=7)

#error@t
model.fit(x,y)

Get 10000 rows from the test dataset (note: this dataset is different from the training dataset (no overlap, and exists in time AFTER the training dataset as to avoid leaking information via the characteristics))

In [ ]:
sql = (
"select "
"top 10000 "
"*, "
"(ORD_LINE_CANC / (ORD_LINE_COMP + ORD_LINE_CANC)) as ORD_CANC_RATIO, "
"(CUST_LINES_CANC / (CUST_LINES_COMP + CUST_LINES_CANC)) as CUST_CANC_RATIO, "
"(PLANT_LINES_CANC / (PLANT_LINES_COMP + PLANT_LINES_CANC)) as PLANT_CANC_RATIO "
"from VW_BWH_TEST_LINE_ORD_CUST_DEPOT "
"where NET_VALUE > '0' "
"ORDER BY RAND() LIMIT 10000"
)

po_sel = conn.sql(sql)
test = po_sel.collect()
#delet

Format

In [ ]:
test['NET_VALUE'] = test['NET_VALUE'].astype(float).astype(int)
test['ORD_NET_VALUE_TOTAL'] = test['ORD_NET_VALUE_TOTAL'].astype(float).astype(int)
test['ORD_NET_VALUE_COMP'] = test['ORD_NET_VALUE_COMP'].astype(float).astype(int)
test['ORD_NET_VALUE_CANC'] = test['ORD_NET_VALUE_CANC'].astype(float).astype(int)
test['CUST_NET_VALUE_TOTAL'] = test['CUST_NET_VALUE_TOTAL'].astype(float).astype(int)
test['CUST_NET_VALUE_COMP'] = test['CUST_NET_VALUE_COMP'].astype(float).astype(int)
test['CUST_NET_VALUE_CANC'] = test['CUST_NET_VALUE_CANC'].astype(float).astype(int)
test['PLANT_NET_VALUE_TOTAL'] = test['PLANT_NET_VALUE_TOTAL'].astype(float).astype(int)
test['PLANT_NET_VALUE_COMP'] = test['PLANT_NET_VALUE_COMP'].astype(float).astype(int)
test['PLANT_NET_VALUE_CANC'] = test['PLANT_NET_VALUE_CANC'].astype(float).astype(int)
test['ORD_CANC_RATIO'] = test['ORD_CANC_RATIO'].astype(float)
test['CUST_CANC_RATIO'] = test['CUST_CANC_RATIO'].astype(float)
test['PLANT_CANC_RATIO'] = test['PLANT_CANC_RATIO'].astype(float)

In [ ]:
y_test = test['COMP_FLAG']
datatest = test.drop(['CANC_FLAG','COMP_FLAG','DOC_NUMBER','S_ORD_ITEM','CUSTOMER','CUST_NAME','CUST_CITY','CUST_POSTAL_CD','PLANT','PLANT_NAME','MATERIAL','ORDSTAT','CREATEDON_SO','CREATEDON_LINE','LINE_COMP_DATE','K8INV_DT','RECORDMODE'], axis=1)
#include region

Apply the SAME label binarizer as was trained on the training dataset

In [ ]:
testtransformed = jobs_encoder.transform(datatest['REGION'])
ohe_dftest = pd.DataFrame(testtransformed, columns=jobs_encoder.classes_)
datatest.reset_index(inplace=True, drop=True)
x_test = pd.concat([datatest, ohe_dftest], axis=1)
x_test = x_test.drop(['REGION'], axis = 1)
x_test.head(30)

Calculate accuracy for the predictions, using the test dataset

In [ ]:
# make predictions for test data
y_pred = model.predict(x_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Plot confusion matrix

In [ ]:
confm = confusion_matrix(y_test, y_pred)

cmd_obj = ConfusionMatrixDisplay(confm, display_labels=[model.classes_[0], model.classes_[1]])
cmd_obj.plot()
cmd_obj.ax_.set(
                title='Confusion Matrix', 
                xlabel='Predicted', 
                ylabel='Actual')
plt.show()

Looks terrible! Lets try to alter the decision threshold to 17%...

In [ ]:
y_pred = (model.predict_proba(x_test)[:,1] >= 0.17).astype(bool) # set threshold as 0.3

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Confusion matrix again. Looks a lot better

In [ ]:
confm = confusion_matrix(y_test, y_pred)

cmd_obj = ConfusionMatrixDisplay(confm, display_labels=[model.classes_[0], model.classes_[1]])
cmd_obj.plot()
cmd_obj.ax_.set(
                title='Confusion Matrix', 
                xlabel='Predicted', 
                ylabel='Actual')
plt.show()

Show the test lines that the model is most certain about will be cancelled or completed respectfully (just change the ascending between True or False...)

In [ ]:
y_test_proba = pd.DataFrame(model.predict_proba(x_test))
comparison = y_test_proba.join(y_test).join(test[['DOC_NUMBER','CANC_FLAG','S_ORD_ITEM']])
comparison = comparison.sort_values(by=comparison.columns[1], ascending = True)
comparison.rename(columns={list(comparison)[0]: 'Proba_Cancel', list(comparison)[1]: 'Proba_Invoice'}, inplace=True)
cols = comparison.columns.tolist()
#['DOC_NUMBER', 'S_ORD_ITEM', 'Proba_Cancel', 'Proba_Invoice', 'COMP_FLAG']
#comparison.head(250)
comparison2 = comparison[['DOC_NUMBER', 'S_ORD_ITEM', 'Proba_Cancel', 'Proba_Invoice', 'COMP_FLAG']]
comparison2.head(250)